# Chat

In [ ]:
# !pip install langchain-openai

In [13]:
from langchain_openai import ChatOpenAI
import os

chat = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0.0)

### What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, 
### while lower values like 0.2 will make it more focused and deterministic.

# print(chat.model_name)
# chat.invoke("Hello, how are you?")

# ChatPromptTemplate

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(
    """Translate the text that is delimited by triple backticks \
    into a style that is {style}. text: ```{text}```"""
)
# prompt_template.messages[0].prompt
# prompt_template.messages[0].prompt.input_variables
# prompts = prompt_template.format_messages(style="", text="")

['style', 'text']

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])
chain = prompt | chat 
chain.invoke({"input": "how can langsmith help with testing?"})

# Parser

In [11]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt = ChatPromptTemplate.from_template(template=template)
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)
response = chat.invoke(messages)
output_dict = output_parser.parse(response.content)
output_dict

# Memory

In [16]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [17]:
llm = ChatOpenAI(temperature=0.0, openai_api_key=os.getenv("OPENAI_API_KEY"))
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [18]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [19]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?


In [20]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?"}

In [25]:
memory = ConversationBufferMemory()
print(memory.buffer, "\n...")
memory.save_context({"input": "Hi"}, {"output": "What's up"})
print(memory.buffer, "\n...")
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
print(memory.buffer, "\n...")

 
...
Human: Hi
AI: What's up 
...
Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool 
...


In [31]:
from langchain.memory import ConversationBufferWindowMemory

### k is the number of previous messages to remember
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [32]:
from langchain.memory import ConversationTokenBufferMemory

### need pip install tiktoken
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})
memory.load_memory_variables({})


{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

In [33]:
from langchain.memory import ConversationSummaryBufferMemory

### this summarize the conversation if conversation exceeds max_token_limit
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, {"output": f"{schedule}"})
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events."}

# Chain

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.9, openai_api_key=os.getenv("OPENAI_API_KEY"))
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
product = "Queen Size Sheet Set"
chain.invoke(product)

In [ ]:
from langchain.chains import SimpleSequentialChain

llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

### this is for single input and single output
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [37]:
from langchain.chains import SequentialChain

llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template("Translate the following review to english:\n\n{Review}")
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")
# prompt template 2: summarize the review in 1 sentence
second_prompt = ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence:\n\n{English_Review}")
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")
# prompt template 3: identify the language of the review
third_prompt = ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}")
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,output_key="language")
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template("Write a follow up response to the following summary in the specified language:\n\nSummary: {summary}\n\nLanguage: {language}")
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,output_key="followup_message")

# overall_chain: input= Review and output= English_Review,summary, followup_message
# here, chains and input_variables are required, output_variables are optional
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "language", "summary","followup_message"],
    verbose=True
)

{'product': 'Queen Size Sheet Set', 'text': '"Regal Rest Linens"'}

# Indexes, Stores, and Loaders

# Agents

# Evaluation